In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import os
import geopandas as gpd
import keras
import gee
import utils


In [11]:
region_name = 'pang_war_myanmar'
#region_name = 'test_region_small'


tile_size = 576 # this is the around the max size that GEE exports allow with 12-band imagery
tile_padding = 24

start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)
clear_threshold = 0.6

pred_threshold = 0.3

region = gpd.read_file(f'../data/boundaries/{region_name}.geojson').geometry[0].__geo_interface__
tiles = utils.create_tiles(region, tile_size, tile_padding)
print(f"Created {len(tiles):,} tiles")

Created 72 tiles


In [12]:
# load an ensembled model

#connect to cpu only
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

model_name = '48px_v3.7-ensemble_2024-01-17'
model = keras.models.load_model(f'../models/{model_name}.h5', compile=False)

In [13]:
import gee
data_pipeline = gee.S2_Data_Extractor(
    tiles, 
    start_date, 
    end_date, 
    clear_threshold, 
    batch_size=10
    )

In [14]:
predictions=data_pipeline.make_predictions(model, pred_threshold=pred_threshold, tries=2)


 12%|█▎        | 1/8 [00:20<02:22, 20.37s/it]

Found 13 positives.


 25%|██▌       | 2/8 [01:08<03:40, 36.81s/it]

Found 32 positives.


 38%|███▊      | 3/8 [01:27<02:22, 28.56s/it]

...

 88%|████████▊ | 7/8 [02:48<00:21, 21.47s/it]

Found 50 positives.


100%|██████████| 8/8 [03:12<00:00, 24.04s/it]


Found 50 positives.


0it [00:00, ?it/s]


In [15]:
# write the predictions to a file
print(len(predictions), 'chips with predictions above', pred_threshold)
# write the predictions to a file
model_version_name = '_'.join(model_name.split('_')[0:2])
# if the outputs directory does not exist, create it
if not os.path.exists(f'../data/outputs/{model_version_name}'):
    os.makedirs(f'../data/outputs/{model_version_name}')
time_period = f"{start_date.month}_{start_date.year}-{end_date.month}_{end_date.year}"
predictions.to_file(f"../data/outputs/{model_version_name}/{region_name}_{model_version_name}_{pred_threshold:.2f}_{time_period}.geojson", driver="GeoJSON")

50 chips with predictions above 0.3


In [36]:
import plotly.express as px

predictions['lat'] = predictions['geometry'].apply(lambda x: x.centroid.y)
predictions['lon'] = predictions['geometry'].apply(lambda x: x.centroid.x)

fig = px.scatter_mapbox(predictions, lat="lat", lon="lon", zoom=10)
fig.update_layout(mapbox_style="satellite-streets", mapbox_accesstoken='pk.eyJ1Ijoib2JhbGxpbmdlciIsImEiOiJjazdvbXFkMWEwYXVrM2ZtdHU0cTd0cG5hIn0.vIj9s8F6M49ZP5DAh9Kd3Q')
fig.show()